# Universidad de Buenos Aires
# Deep Learning - TP1
# Noviembre 2023



El TP comienza al momento de recibir este correo y la ventana de entrega estará abierta hasta el Domingo 24 de diciembre. La resolución es individual. Pueden utilizar los contenidos vistos en clase y otra bibliografía. Si se toman ideas de fuentes externas deben ser correctamente citas incluyendo el correspondiente link o página de libro.

El formato de entrega debe ser un “link a un colab” (compartir a las siguientes direcciones: maxit1992@gmail.com y lelectronfou@gmail.com). Tanto los resultados, como el código y las explicaciones deben quedar guardados y visualizables en el colab.

## Ejercicio 1



Se quiere encontrar el máximo de la siguiente función:

$z = -(x - 2)^2 - (y - 3)^2 + 4$
<br>
<br>
1. Aplicar gradiente de forma analítica e igualar a zero para encontrar los valores de $x$ e $y$ donde $z$ tiene un máximo. Cuál es el valor del máximo?

2. Aplicar SGD para encontrar la ubicación del máximo de manera numérica (pueden utilizar pytorch). Comparar con el resultado obtenido en el punto 1

## Ejercicio 2


Descargar el dataset del siguiente link: https://drive.google.com/file/d/1eFWn7eDmSFUK1JuuBBykxkC9J0CGYDKe/view?usp=sharing.

El dataset contiene mediciones obtenidas al ensayar un sistema de posicionamiento. El sistema consiste en un dispositivo móvil del cual se desea conocer la posición y 13 "balizas" fijas (distribuidas en un salón) que emiten señales de radio.

Cada fila del dataset contiene una posición del dispositivo móvil y los niveles de señal recibida (de las señales emitidas por cada una de las 13 balizas fijas) en dicha posición.

![Salon](https://drive.google.com/uc?export=view&id=1z3uHEd3tS1kQpGXfhPYn2GFfA95v_ArW)


Algunas consideraciones:
- La imágen anterior es orientativa, no se encuentra a escala ni representa la verdadera posición de las balizas fijas.
- La posición en el salón se divide en una cuadrícula. La posición horizontal se codifica con una letra de la A a la Z y la posición vertical se codifica con valores de 01 a 20.
- El nivel de señal recibida se mide de 0 (máximo teórico) a -200 (mínimo teórico). NA significa que no se recibe señal de la baliza en dicha posición. A efectos prácticos no recibir señal (NA) es equivalente a recibir una señal con nivel -200.

**Consignas:**

1. Analizar el dataset y aplicar las transformaciones que considere necesarias para entrenar un modelo de red neuronal.

2. Entrenar un modelo de **Deep Learning** con múltiples capas lineales que prediga la posición del dispositivo móvil en el salón (vertical y horizontal) a partir de las mediciones de los niveles de las 13 balizas. Graficar la evolución de la función de pérdida y la evolución de la métrica [MAE](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html) durante el entrenamiento.

3. Comprobar el funcionamiento del modelo realizando una predicción sobre una muestra aleatoria del dataset y comparar con la posición real.

Con la finalidad de ahorrar energía en el dispositivo móvil y simplificar el sistema, se quiere ensayar la posibilidad de predecir la posición solamente con la información del nivel de señal de las 2 balizas mas cercanas.

4. Aplicar las transformaciones necesarias sobre el dataset para obtener un nuevo dataset que contenga solamente la información de las 2 balizas con mayor nivel de señal (ver imágen adjunta). Si no se recibe señal de una 2da baliza, proponer un método para completar la información faltante.

![Dataset Punto 4](https://drive.google.com/uc?export=view&id=1kz1Y5m5rmbYPiuZIc4QHvnt4uFB2TwWu)


5. Entrenar un modelo de **Deep Learning** que prediga la posición del dispositivo móvil en el salón (vertical y horizontal) a partir del dataset del punto 4, incluyendo **una capa de embeddings** para ambos número (o IDs) de balizas.

6. Comparar los resultados obtenidos con los modelo de los puntos 2 y 5 y enunciar conclusiones.

# Resolucion
## Importacion de librerías públicas

In [ ]:
# Importar librerías
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from numpy import random, vectorize
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, Dataset

## Asignar cómputo a la gráfica o a la CPU

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## Definición de funciones

In [ ]:
def process_single_coordinate(coor: str):
    ZERO_LETTER = ord("A")
    x = ord(coor[0]) - ZERO_LETTER
    y = int(coor[1:3])
    return x, y


def process_coordinates(col: pd.Series):
    x, y = process_single_coordinate(col)
    return pd.Series({"x": x, "y": y})

## Ejercicio 1
#### Apartado 1
La resolución analítica es la siguiente:

z = -(x-2)^2 -(y-3)^2 + 4 

z = -x^2 + 4x - 4 - y^2 + 6y - 9 + 4

z = -x^2 + 4x - y^2 + 6y -9


Realizamos el cáculo del gradiente derivando la función en `x` y en `y`:

dz/dx = -2x + 4

dz/dy = -2y + 6


El punto donde el gradiente es 0 es en el punto (x,y) = (2,3)

#### Apartado 2

In [ ]:
# Defino 2 tensores con seguimiento de gradiente, valor inicial arbitrario
x = torch.tensor([4.0], requires_grad=True)
y = torch.tensor([-1.0], requires_grad=True)

# Hiperparámetros SGD
lr = 0.1
max_iter = 50

# Resultados intermedios
x_values = []
y_values = []
z_values = []

# SGD Loop
for epoch in range(max_iter):
    z = (
        (x - 2) ** 2 + (y - 3) ** 2 - 4
    )  # Buscar un máximo de z es lo mismo que buscar el mínimo de -z

    # Guardar los valores en cada iteración
    x_values.append(x.item())
    y_values.append(y.item())
    z_values.append(-1 * z.item())  # El valor de z sí es el original

    z.backward()  # Calcular gradientes

    # Actualizar parámetros
    with torch.no_grad():
        x -= lr * x.grad
        y -= lr * y.grad
        # Reiniciar gradientes
        x.grad.zero_()
        y.grad.zero_()

# Impresión de resultados
plt.figure(figsize=(12, 4))

plt.plot(x_values, label="x")
plt.plot(y_values, label="y")
plt.plot(z_values, label="z")

plt.title("Evolución de x, y, y z en cada iteración")
plt.xlabel("Iteración")
plt.legend()

plt.show()

Como se puede observar, el método de gradiente actualiza los valores de las variables hasta que alcanzan el punto de gradiente 0 en el punto (2,3)
tal y como habíamos calculado de forma analítica

## Ejercicio 2
#### Apartado 1

Transformaciones aplicadas al set de datos:
- Convertir los datos faltantes en -200
- Transformar las coordenadas en x,y para la función distancia

In [ ]:
data = pd.read_csv("resources/Positioning_data.csv")
data = data.fillna(-200)  # Reemplazo de valores vacíos
data[["x", "y"]] = data["Pos"].apply(process_coordinates)

# Preparación de datos
x_cols = [
    "Baliza1",
    "Baliza2",
    "Baliza3",
    "Baliza4",
    "Baliza5",
    "Baliza6",
    "Baliza7",
    "Baliza8",
    "Baliza9",
    "Baliza10",
    "Baliza11",
    "Baliza12",
    "Baliza13",
]
y_cols = ["x", "y"]

X_e = data[x_cols].values
Y_e = data[y_cols].values

print(data[y_cols+x_cols].head())

#### Apartado 2
Definición de la red neuronal

In [ ]:
class BeaconPositioningNN(nn.Module):
    layer_1: nn.Linear
    layer_2: nn.ReLU
    layer_3: nn.Linear
    layer_4: nn.ReLU
    layer_5: nn.Linear
    output: nn.LeakyReLU

    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(
            in_features=13, out_features=78, bias=True
        )  # Idea for number of perceptrons in middle layers: nº of relations between 13 beacons = 13*12/2
        self.layer_2 = nn.ReLU()
        self.layer_3 = nn.Linear(in_features=78, out_features=78, bias=True)
        self.layer_4 = nn.ReLU()
        self.layer_5 = nn.Linear(in_features=78, out_features=2, bias=True)
        self.output = nn.LeakyReLU()

    def forward(self, x):
        # Defino el cálculo del paso forward
        x = self.layer_1(x)
        x = self.layer_2(x)
        x = self.layer_3(x)
        x = self.layer_4(x)
        x = self.layer_5(x)
        x = self.output(x)
        return x

Resolución

In [ ]:
# Dividir datos en conjuntos de entrenamiento y prueba
X_train_e_num, X_test_e_num, Y_train_e, Y_test_e = train_test_split(
    X_e, Y_e, test_size=0.2, random_state=42
)

# Convertir a tensores de PyTorch
X_train_e_num = torch.tensor(X_train_e_num, dtype=torch.float32).to(device)
Y_train_e = torch.tensor(Y_train_e, dtype=torch.float32).to(device)
X_test_e_num = torch.tensor(X_test_e_num, dtype=torch.float32).to(device)
Y_test_e = torch.tensor(Y_test_e, dtype=torch.float32).to(device)

# Instanciamos la red
nnet_e = BeaconPositioningNN()
# Copio la red neuronal al dispositivo donde entrene la red neuronal
nnet_e = nnet_e.to(device)
# Mi funcion de Loss es MSE porque es simplemente la versión promediada de la distancia euclidiana al cuadrado
loss_function = torch.nn.MSELoss()
mae_function = mean_absolute_error
# Optimizer
optimizer = torch.optim.Adam(nnet_e.parameters(), lr=0.0001)
# cantidad de epochs
epochs = 1000

# Lista para almacenar el valor medio de pérdida en cada iteración
train_loss_values = []
valid_loss_values = []
train_mae_values = []
valid_mae_values = []

# Doble loop algoritmo Mini-Batch
for epoch in range(epochs):
    ############################################
    ## Entrenamiento
    ############################################
    nnet_e.train(True)

    # Paso forward
    # Limpio optimizer para empezar un nuevo cálculo de gradiente
    optimizer.zero_grad()
    train_output = nnet_e(X_train_e_num)

    # Calculo el loss y mae
    train_loss = loss_function(train_output, Y_train_e)
    train_mae = mae_function(train_output.cpu().detach(), Y_train_e.cpu().detach())

    # Backpropagation
    train_loss.backward()

    # Actualizar los parámetros
    optimizer.step()

    # Calcular y almacenar el valor medio
    train_avg_loss = torch.mean(train_loss).item()
    train_loss_values.append(train_avg_loss)
    train_mae_values.append(train_mae)

    ############################################
    ## Validación
    ############################################
    # Desactivo el cálculo de gradiente para validación
    nnet_e.train(False)

    # Paso forward
    valid_output = nnet_e(X_test_e_num)

    # Calculo el loss
    valid_loss = loss_function(valid_output, Y_test_e)
    valid_mae = mae_function(valid_output.cpu().detach(), Y_test_e.cpu().detach())

    # En validación no hago backpropagation!!

    # Calcular y almacenar el valor medio de pérdida
    valid_avg_loss = torch.mean(valid_loss).item()
    valid_loss_values.append(valid_avg_loss)
    valid_mae_values.append(valid_mae)

############################################
## Impresión de resultados por epoch
############################################

# Crear una figura con dos subgráficos (2 filas, 1 columna)
fig, axes = plt.subplots(1, 2, figsize=(20, 8))
# Graficar
axes[0].plot(train_loss_values, label="train")
axes[0].plot(valid_loss_values, label="validation")
axes[0].set_xlabel('Iteración')
axes[0].set_ylabel('Loss')
axes[0].set_title('Loss function (MSE) evolution')
axes[0].legend()

axes[1].plot(train_mae_values, label="train")
axes[1].plot(valid_mae_values, label="validation")
axes[1].set_xlabel('Iteración')
axes[1].set_ylabel('MAE')
axes[1].set_title('MAE evolution')
axes[1].legend()

# Agregar etiquetas y título
fig.suptitle("Metricas")


# Mostrar el gráfico
plt.show()

#### Apartado 3

In [ ]:
random_index = random.choice(len(X_e), size=5, replace=False)

ran_x = torch.tensor(X_e[random_index], dtype=torch.float32).to(device)
ran_y = Y_e[random_index]

nnet_e.train(False)

pred_y = nnet_e(ran_x).cpu().detach()

# Crear un gráfico de dispersión para visualizar las predicciones y los valores reales
plt.scatter(
    pred_y[:, 0],
    pred_y[:, 1],
    label="Predicciones",
    c=range(len(pred_y)),
    cmap="cividis",
)
plt.scatter(
    ran_y[:, 0],
    ran_y[:, 1],
    label="Valores reales",
    c=range(len(ran_y)),
    cmap="cividis",
    marker="x",
)

# Etiquetas y título del gráfico
plt.xlabel("Valores reales")
plt.ylabel("Predicciones")
plt.title("Comparación entre valores reales y predicciones")
plt.xticks(range(26), [chr(65 + i) for i in range(26)])  # Añadir etiquetas de la A a la Z
plt.legend()

# Mostrar la gráfica
plt.show()

#### Apartado 4

In [ ]:
# Crear un nuevo DataFrame con las columnas x e y
data_embedded = data[y_cols].copy()

# Iterar sobre las filas del DataFrame original
for index, row in data.iterrows():
    # Obtener las columnas de las balizas originales para la fila actual
    balizas = row[x_cols]

    # Encontrar las dos balizas más cercanas y sus valores
    baliza_cercana_1 = balizas.idxmax()
    valor_cercano_1 = balizas.max()

    # Eliminar la baliza más cercana de las balizas originales y encontrar la segunda baliza más cercana
    balizas = balizas.drop(baliza_cercana_1)
    baliza_cercana_2 = balizas.idxmax()
    valor_cercano_2 = balizas.max()

    if valor_cercano_2 == -200:
        baliza_cercana_2 = "Ninguna"

    # Agregar las columnas al nuevo DataFrame
    data_embedded.loc[
        index,
        ["Baliza_Cercana_1", "Valor_Cercano_1", "Baliza_Cercana_2", "Valor_Cercano_2"],
    ] = [
        str(baliza_cercana_1),
        float(valor_cercano_1),
        str(baliza_cercana_2),
        float(valor_cercano_2),
    ]

# Mostrar el nuevo conjunto de datos
print(data_embedded.head(12))

#### Apartado 5
Nueva clase

In [ ]:
class EmbeddedBeaconPositioningNN(nn.Module):
    embedding: nn.Embedding
    layer_1: nn.Linear
    layer_2: nn.ReLU
    layer_3: nn.Linear
    layer_4: nn.ReLU
    layer_5: nn.Linear
    output: nn.LeakyReLU

    def __init__(self):
        super().__init__()
        self.embedding = self.embedding_layer = nn.Embedding(14, 13)
        self.layer_1 = nn.Linear(
            in_features=28, out_features=78, bias=True
        )  # Idea for number of perceptrons in middle layers: nº of relations between 13 beacons = 13*12/2
        self.layer_2 = nn.ReLU()
        self.layer_3 = nn.Linear(in_features=78, out_features=78, bias=True)
        self.layer_4 = nn.ReLU()
        self.layer_5 = nn.Linear(in_features=78, out_features=2, bias=True)
        self.output = nn.LeakyReLU()

    def forward(self, x, baliza_cercana_1, baliza_cercana_2):
        # Paso las balizas cercanas a través de las capas de embedding
        embedded_baliza_cercana_1 = self.embedding_layer(baliza_cercana_1).squeeze()
        embedded_baliza_cercana_2 = self.embedding_layer(baliza_cercana_2).squeeze()

        # Concateno las embeddings con las características originales
        x = torch.cat([x, embedded_baliza_cercana_1, embedded_baliza_cercana_2], dim=1)
        
        # Defino el cálculo del paso forward
        x = self.layer_1(x)
        x = self.layer_2(x)
        x = self.layer_3(x)
        x = self.layer_4(x)
        x = self.layer_5(x)
        x = self.output(x)
        return x

Resolución

In [ ]:
baliza_mapping = {baliza: idx for idx, baliza in enumerate(x_cols + ["Ninguna"])}

X_e = data_embedded[
    ["Baliza_Cercana_1", "Valor_Cercano_1", "Baliza_Cercana_2", "Valor_Cercano_2"]
]
Y_e = data_embedded[y_cols]
# Dividir datos en conjuntos de entrenamiento y prueba
X_train_e, X_test_e, Y_train_e, Y_test_e = train_test_split(
    X_e, Y_e, test_size=0.2, random_state=42
)

# Convertir a tensores de PyTorch
X_train_e_num = torch.tensor(
    X_train_e[["Valor_Cercano_1", "Valor_Cercano_2"]].values, dtype=torch.float32
).to(device)
Y_train_e = torch.tensor(Y_train_e.values, dtype=torch.float32).to(device)
X_test_e_num = torch.tensor(
    X_test_e[["Valor_Cercano_1", "Valor_Cercano_2"]].values, dtype=torch.float32
).to(device)
Y_test_e = torch.tensor(Y_test_e.values, dtype=torch.float32).to(device)

# Instanciamos la red
nnet_e = EmbeddedBeaconPositioningNN()
# Copio la red neuronal al dispositivo donde entrene la red neuronal
nnet_e = nnet_e.to(device)

mapping = vectorize(lambda x: baliza_mapping[x])

# Optimizer
optimizer_e = torch.optim.Adam(nnet_e.parameters(), lr=0.0001)
# cantidad de epochs
epochs = 1000

# Lista para almacenar el valor medio de pérdida en cada iteración
train_loss_values_e = []
valid_loss_values_e = []
train_mae_values_e = []
valid_mae_values_e = []

# Doble loop algoritmo Mini-Batch
for epoch in range(epochs):
    ############################################
    ## Entrenamiento
    ############################################
    nnet_e.train(True)

    # Paso forward
    # Limpio optimizer para empezar un nuevo cálculo de gradiente
    optimizer_e.zero_grad()

    train_col_bal_1 = X_train_e[["Baliza_Cercana_1"]].values
    train_col_bal_2 = X_train_e[["Baliza_Cercana_2"]].values

    train_bal_1 = torch.tensor(mapping(train_col_bal_1), dtype=torch.long).to(device)
    train_bal_2 = torch.tensor(mapping(train_col_bal_2), dtype=torch.long).to(device)

    train_output = nnet_e(X_train_e_num, train_bal_1, train_bal_2)

    # Calculo el loss y mae
    train_loss = loss_function(train_output, Y_train_e)
    train_mae = mae_function(train_output.cpu().detach(), Y_train_e.cpu().detach())

    # Backpropagation
    train_loss.backward()

    # Actualizar los parámetros
    optimizer_e.step()

    # Calcular y almacenar el valor medio
    train_avg_loss = torch.mean(train_loss).item()
    train_loss_values_e.append(train_avg_loss)
    train_mae_values_e.append(train_mae)

    ############################################
    ## Validación
    ############################################
    # Desactivo el cálculo de gradiente para validación
    nnet_e.train(False)

    valid_col_bal_1 = X_test_e[["Baliza_Cercana_1"]].values
    valid_col_bal_2 = X_test_e[["Baliza_Cercana_2"]].values
    valid_bal_1 = torch.tensor(mapping(valid_col_bal_1), dtype=torch.long).to(device)
    valid_bal_2 = torch.tensor(mapping(valid_col_bal_2), dtype=torch.long).to(device)

    # Paso forward
    valid_output = nnet_e(X_test_e_num, valid_bal_1, valid_bal_2)

    # Calculo el loss
    valid_loss = loss_function(valid_output, Y_test_e)
    valid_mae = mae_function(valid_output.cpu().detach(), Y_test_e.cpu().detach())

    # En validación no hago backpropagation!!

    # Calcular y almacenar el valor medio de pérdida
    valid_avg_loss = torch.mean(valid_loss).item()
    valid_loss_values_e.append(valid_avg_loss)
    valid_mae_values_e.append(valid_mae)

############################################
## Impresión de resultados por epoch
############################################

# Crear una figura con dos subgráficos (2 filas, 1 columna)
fig, axes = plt.subplots(1, 2, figsize=(20, 8))
# Graficar
axes[0].plot(train_loss_values, label="train")
axes[0].plot(valid_loss_values, label="validation")
axes[0].plot(train_loss_values_e, label="train(embedded)")
axes[0].plot(valid_loss_values_e, label="validation(embedded)")
axes[0].set_xlabel("Iteración")
axes[0].set_ylabel("Loss")
axes[0].set_title("Loss function (MSE) evolution")
axes[0].legend()

axes[1].plot(train_mae_values, label="train")
axes[1].plot(valid_mae_values, label="validation")
axes[1].plot(train_mae_values_e, label="train(embedded)")
axes[1].plot(valid_mae_values_e, label="validation(embedded)")
axes[1].set_xlabel("Iteración")
axes[1].set_ylabel("MAE")
axes[1].set_title("MAE evolution")
axes[1].legend()

# Agregar etiquetas y título
fig.suptitle("Metricas")


# Mostrar el gráfico
plt.show()

#### Apartado 6
Tal y como se muestra en las gráficas, se aprecia que la red neuronal con embedding requiere un mayor número de épocas para entrenar. El tiempo de procesamiento tambien es ligeramente mayor que el de la otra red con el modelo de datos que manejamos.

La ventaja del modelo con embeddings es que se reduce en gran cantidad la cantidad de datos necesarios, sobre todo si casi nunca se obtiene señal de más de una baliza y en el supuesto de que el número de balizas aumente considerablemente.